In [ ]:
import os
import sys

from matplotlib import pyplot as plt
import matplotlib.patches as mpatch
import numpy as np
from scipy.integrate import dblquad
from scipy.stats import multivariate_normal

CWD = os.path.abspath("")
sys.path.append(os.path.join(CWD, ".."))
from plt_settings import plt_settings

full_width = 5.5
ratio = 1 / 1.618

In [ ]:
ITERATIONS = int(1e5)

sampling_data = np.load(os.path.join(CWD, "2d_gaussian_vs_N.npz"))

pdlmc_samples = {}
pdlmc_samples[1] = sampling_data["pdlmc_x_1"]
pdlmc_samples[10] = sampling_data["pdlmc_x_10"]
pdlmc_samples[100] = sampling_data["pdlmc_x_100"]
pdlmc_samples[1000] = sampling_data["pdlmc_x_1000"]

pdlmc_lambda = {}
pdlmc_lambda[1] = sampling_data["pdlmc_lambda_1"]
pdlmc_lambda[10] = sampling_data["pdlmc_lambda_10"]
pdlmc_lambda[100] = sampling_data["pdlmc_lambda_100"]
pdlmc_lambda[1000] = sampling_data["pdlmc_lambda_1000"]

In [ ]:
# True mean
Z, _ = dblquad(
    lambda y, x: multivariate_normal.pdf([x, y], mean=np.ones(2) * 2),
    -1,
    1,
    lambda x: -np.sqrt(1 - x**2),
    lambda x: np.sqrt(1 - x**2),
)
true_mean_x, _ = dblquad(
    lambda y, x: x * multivariate_normal.pdf([x, y], mean=np.ones(2) * 2),
    -1,
    1,
    lambda x: -np.sqrt(1 - x**2),
    lambda x: np.sqrt(1 - x**2),
)

true_mean = np.array([true_mean_x / Z, true_mean_x / Z])
print(f"Mean of the distribution: {true_mean}")

print("")

# Estimated mean
for N, samples in pdlmc_samples.items():
    print(
        f"[N = {N:>4}] Estimated mean: {samples[:ITERATIONS // N].mean(axis=0)}",
        flush=True,
    )

print("")

# Out-of-support samples
for N, samples in pdlmc_samples.items():
    out_of_support = (
        (samples[: ITERATIONS // N] * samples[: ITERATIONS // N]).sum(axis=1) > 1
    ).mean()
    print(f"[N = {N:>4}] Percentage of out-of-support samples: {out_of_support * 100:.2f}%")

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, full_width / 2)
scatter_kwargs = {
    "s": 1.5,
    "edgecolors": "face",
    "alpha": 0.5,
    "linewidth": 0.0,
}

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    for N, samples in pdlmc_samples.items():
        cum_mean = np.cumsum(samples, axis=0) / np.expand_dims(np.arange(1, ITERATIONS + 1), 1)
        axs.scatter(
            cum_mean[: ITERATIONS // N, 0],
            cum_mean[: ITERATIONS // N, 1],
            label=f"N = {N}",
            **scatter_kwargs,
        )
    axs.scatter([true_mean[0]], [true_mean[1]], marker="+", color="k", label="True mean")
    axs.add_artist(mpatch.Circle((0.0, 0.0), 1.0, facecolor="none", edgecolor="k", lw=0.7))
    axs.set_xlim((0, 1.0))
    axs.set_ylim((0, 1.0))
    axs.set_xticks([])
    axs.set_yticks([])
    axs.set_aspect(1)
    plt.legend()

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    for N, lmbdas in pdlmc_lambda.items():
        axs.plot(N * np.arange(ITERATIONS // N), lmbdas[: ITERATIONS // N], label=rf"$N_b = {N}$")
    axs.ticklabel_format(scilimits=(0, 2))
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_ylabel(r"Dual variable ($\lambda$)")
    plt.legend(loc="upper left")

    plt.show()